# My take in the coursera assigment 'Segmenting and Clustering Neighboorhoods in Toronto' from the Applied Data Science Capstone course

In [1]:
import pandas as pd

## Part 1 (Recap) - Scrape the Neighbourhood data off of wikipedia and clean it up

See Part 2 below!


### Read the data into a pandas dataframe

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Simply employ the pandas `read_html()` option to scrape the tabular data from the wikipedia page.
This method scrapes every tabular content into a list of tabular objects. We are interested in the first table on the website, e.g., the zip codes of Toronto

In [11]:
tables = pd.read_html(wiki_url)
zipcode_df=tables[0]
zipcode_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean the data

1. Delete all rows for which the Borough is 'Not assigned'
2. Combine neighbourhoods with the same postcode
3. Replace the Neighbourhood names that are 'Not assigned' with the borough name

In [12]:
zipcode_df = zipcode_df[zipcode_df.Borough != 'Not assigned']
zipcode_df = zipcode_df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
zipcode_df.Neighborhood[zipcode_df.Neighborhood == 'Not assigned'] = zipcode_df.Borough
#zipcode_df.head()

## Part 2 - combine the dataframe from Part 1 with geolocation data

### This part is commented out, because it does not work. So, i downloaded the .csv file with Tor

`
import geocoder # import geocoder

toronto_zipcodes = zipcode_df['Postcode'].values
len(toronto_zipcodes)
latitudes = longitudes = []


for i, postal_code in enumerate(toronto_zipcodes):
    # initialize your variable to None
    lat_lng_coords = None
    gurke = 0

    # loop until you get the coordinates
    while(lat_lng_coords is None and gurke < 10):
        gurke += 1
        g = geocoder.google('ontario, {}, canada'.format(postal_code))
        lat_lng_coords = g.latlng
        print(postal_code, lat_lng_coords)
        
    if lat_lng_coords is not None:
        latitudes.append(lat_lng_coords[0])
        longitudes.append(lat_lng_coords[1])
    else:
        latitudes.append(0)
        longitudes.append(0)
`

In [13]:
gsc_file = 'Geospatial_Coordinates.csv' # geospatial coordinates file

Load the .csv file

In [14]:
gsc_pd = pd.read_csv(gsc_file, index_col = 0)
gsc_pd.head(10)

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


Print the dataframe shapes to see if they match!

In [15]:
print(gsc_pd.shape, zipcode_df.shape)

(103, 2) (103, 3)


In [17]:
zipcode_df.set_index('Postcode', inplace = True)

The dataframes can be concatenated using the pd.concat() function:

In [18]:
postcode_with_gsc_pd = pd.concat([zipcode_df, gsc_pd], axis = 1) 
postcode_with_gsc_pd.reset_index(inplace = True)
postcode_with_gsc_pd.rename(columns={"index": "Postcode"}, inplace = True)
postcode_with_gsc_pd.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
postcode_with_gsc_pd.shape

(103, 5)